## Building the Data Pipeline
In this notebook, we'll be building a data pipeline to preprocess and clean the farm survey data. The data includes various geographic, weather, soil, and crop management features for different fields.

Project Structure
The project directory structure is as follows:

- `model/`: Directory containing the notebook for building predictive models.
- `script/`: Directory containing Python scripts for data preprocessing.
- `data/`: Directory containing raw data files and the SQLite database.

## Preprocessing Steps

### 1. Import Required Libraries

We begin by importing the necessary Python libraries for data preprocessing.

```python
import re
import numpy as np
import pandas as pd
import os
import sys
import logging
from preprocessing import FieldDataProcessor


In [9]:
import re
import numpy as np
import pandas as pd
import os
import sys
import logging
from preprocessing import FieldDataProcessor

### 2. Set up Project Directory Paths
We define the paths to the project directories, including the data directory where our raw data is stored.

In [10]:
notebook_dir = os.getcwd()
project_dir = os.path.dirname(notebook_dir)
data_dir = os.path.join(project_dir, 'data')

### 3. Configure Database Connection
We specify the database URL for connecting to the SQLite database file containing the farm survey data.

In [11]:
db_file_path = os.path.join(data_dir, 'Maji_Ndogo_farm_survey_small.db')
db_url = f"sqlite:///{db_file_path}"


### 4. Define Preprocessing Parameters
We set up configuration parameters for preprocessing, including SQL queries, file paths, and regex patterns.

In [13]:
config_params = {
    "sql_query": """
            SELECT *
            FROM geographic_features
            LEFT JOIN weather_features USING (Field_ID)
            LEFT JOIN soil_and_crop_features USING (Field_ID)
            LEFT JOIN farm_management_features USING (Field_ID)
            """,
    "db_path": db_url,
    "columns_to_rename": {'Annual_yield': 'Crop_type', 'Crop_type': 'Annual_yield'},
    "values_to_rename": {'cassaval': 'cassava', 'wheatn': 'wheat', 'teaa': 'tea'},
    "weather_csv_path": os.path.join(data_dir, "Weather_station_data.csv"),
    "weather_mapping_csv": os.path.join(data_dir, "Weather_data_field_mapping.csv"),
    "regex_patterns" : {
            'Rainfall': r'(\d+(\.\d+)?)\s?mm',
            'Temperature': r'(\d+(\.\d+)?)\s?C',
            'Pollution_level': r'=\s*(-?\d+(\.\d+)?)|Pollution at \s*(-?\d+(\.\d+)?)'
            },
}


### 5. Preprocess the Data
We instantiate a FieldDataProcessor object with the configuration parameters and perform the data preprocessing steps.

In [14]:
field_processor = FieldDataProcessor(config_params)
field_processor.process()
field_df = field_processor.df


2024-03-30 12:31:37,311 - data_ingestion - INFO - Database engine created successfully.
2024-03-30 12:31:37,401 - data_ingestion - INFO - Query executed successfully.
2024-03-30 12:31:37,402 - preprocessing.FieldDataProcessor - INFO - Sucessfully loaded data.
2024-03-30 12:31:37,405 - preprocessing.FieldDataProcessor - INFO - Swapped columns: Annual_yield with Crop_type
2024-03-30 12:31:37,417 - data_ingestion - INFO - CSV data ingested successfully.


### 6. Save Cleaned Data
Finally, we save the preprocessed DataFrame to a CSV file in the data directory.

In [15]:
output_csv_path = os.path.join(data_dir, 'clean_field_data.csv')
field_df.to_csv(output_csv_path, index=False)
